In [20]:
# NOTE: The first few cells of this notebook are an algorithm to verify that a triplet of braids
# pairwise glue to be unlinks -- in other words that the triplets form a valid tri-plane diagram
# of a knotted surface in S^4. The third cell describes a high-level overview of the algorithm
# to perform this, and the fifth cell (and onward) describe computing the actual Alexander invariant.



# This function records necessary data for all crossings along a single strand in one braid,
# e.g. for going from the equator to the top of a braid, or from the top of a braid back to the equator.
# Specifically, it keeps track of unique crossings and relevant orientation data to create the unlink later on.
#
# INPUTS:
#     - braid -- The braid word, consisting of numbers -2b <= i <= 2b.
#     - curr_strand -- An integer indicating which strand of the braid is being traversed.
#     - crossing_num -- An integer indicating the number of unique crossings visited.
#     - crossing_number -- A list containing unique crossing numbers for each symbol in
#                          the braid word, or -1 if it has not been encountered already.
#     - orientation -- A list with entries corresponding to each crossing number indicating
#                      if it is a right- or left-hand crossing.
#     - curr_component -- List containing crossings found specifically on the current component of unlink.
#     - flip -- True if the braid word must be traversed in reverse order. This is true when travelling
#               'down' the mirror image braid or 'up' the normal braid.
#     - flip_orientation -- True if we are moving 'up' in the glued unlink, False if down; relevant only for orientation.
#
# OUTPUTS:
#     Returns the strand that we land on after traversal and the number of unique crossings encountered.
def parse_braid(braid, curr_strand, crossing_num, crossing_number, orientation, curr_component, flip, flip_orientation):
    # If we are traversing the braid opposite to its 'normal' direction, we must read the symbols in reverse order.
    if flip:
        braid = braid[::-1]

    # Go through each element of the braid word, keeping track of its index.
    for i, c in enumerate(braid):
        # Since we reversed the braid with [::-1], we must also reverse the index here.
        # We must also change the crossing from over to under (or vice versa); this is just
        # to account for the change of direction relative to traversal along the strand.
        if flip:
            index = len(braid) - i - 1
            c *= -1
        else:
            # If we traversing the braid in normal order, there are no additional steps.
            index = i
            
        # If the braid symbol is adjacent to the current strand, then this symbol affects the traversal.
        if abs(c) == curr_strand or abs(c) == curr_strand + 1:
            # If we haven't seen this crossing yet, give it a unique number.
            if crossing_number[index] == -1:
                crossing_number[index] = crossing_num
                crossing_num += 1
                
            # Once we know the element affects us, we figure out if we're on the overstrand or understrand.
            if c == curr_strand + 1 or (c * -1 == curr_strand): # On the overstrand
                curr_component.append(abs(crossing_number[index])) # Indicate crossing in gauss code
                if c > 0:
                    curr_strand += 1 # If we're on the overstrand and it's positive, we move to the right
                else:
                    curr_strand -= 1 # Otherwise we move to the left
                orientation[crossing_number[index]-1][0] = flip_orientation # Assign relevant orientation information
            else: # Otherwise we're on the understrand
                curr_component.append(abs(crossing_number[index]) * -1) # Indicate crossing in gauss code
                if c > 0:
                    curr_strand -= 1
                    orientation[crossing_number[index] - 1][1] = -1 # understrand to the left
                else:
                    curr_strand += 1
                    orientation[crossing_number[index] - 1][1] = 1  # understrand to the right
    return curr_strand, crossing_num

In [21]:
# This is, as far as I can tell, the simplest way to determine the orientation of a crossing.
# In the extended gauss code, each cross has +1/-1 associated with it, determined as follows:
# Starting at outgoing direction in the overcrossing, find which way to rotate to turn towards
# the outgoing direction of the undercrossing. Clockwise -> -1, counter-clockwise -> +1.
# There's 8 different crossings, and you can map them to +1/-1 based only on if the over-
# crossing goes up or down, and if the under crossing goes left or right.
#
# INPUT:
#     - orientation -- Orientation contains two components: the first is true if the crossing
#                          was visited while travelling up the strand; false otherwise. The
#                          second is 1 if the understrand points to the right, -1 if to the left.
# OUTPUT:
#     - returns the orientation of the crossing as necessary for the extended Gauss code.
def get_orientation(orientation):
    is_up, under = orientation
    if is_up:
        return 1 if under == -1 else -1
    else:
        return under

In [22]:
# Given two braids, e.g. two of the three subdivisions of a tri-plane diagram, compute the
# unlink (union of disjoint knots) created by gluing one braid with the mirror image of the other
# braid. This Link is generated using the Oriented Gauss Code format, as described at:
# https://doc.sagemath.org/html/en/reference/knots/sage/knots/link.html.
# The fact that this always generates an unlink was proven in Meier and Zupan, 2015.
#
# At a high level, the algorithm can be described as follows:
#
# while some strands have not been visited:
#     Pick a strand that has not been visited already
#     while True:
#         Travel down strand, accounting for any not-previously-visited crossings
#         Travel down strand in mirror image in which it was glued to, which also must not have been visited
#         Use braid relation that "caps off" adjacent strands to move to adjacent strand in mirror braid
#         Travel up strand in mirror image, accounting for not-previously-visited crossings
#         Travel up strand in regular braid, accounting for new crossings
#         Use braid relation to move to adjacent strand in regular braid
#         If we are moved to a previously-visited strand, we have completed Link component; BREAK
#         Else, we are still traversing a single link component; CONTINUE
#
#
# INPUT:
#    - braid1 -- The first braid, consisting of numbers -2b <= i <= 2b representing crossings of adjacent strands.
#    - braid2 -- The second braid, consisting of numbers -2b <= i <= 2b representing crossings of adjacent strands.
#    - b -- The bridge number of the tri-plane diagram that the braids are a part of. Also half the number of
#                strands contained in each braid.
#
# OUTPUT:
#    - A sage Link object representing the union of the first braid and mirror image of the second braid.
def glue_braids(braid1, braid2, b):
    # This keeps track of which strands in each braid have and have not been used.
    used_strands = [False] * (2 * b)
    
    # This will contain two lists, each the same length as the two braid words passed in.
    # It associates a unique number to each crossing (or element in either braid word)
    # so that each time we reach a crossing, we can tell if we have visited it already.
    crossing_number = []
    for braid in [braid1, braid2]:
        # Keep track of the crossing number we assign to each element in the braid word
        # It starts with all elements having an associated crossing number of -1, but these
        # are set to the correct number in parse_braid when they are reached.
        crossing_number.append([-1] * len(braid))
    
    # Contains the gauss code. Starting on an arbitrary strand, keep track of the crossings
    # you go past with a number, positive if you went on the overstrand, and negative if
    # you went on the understrand. Once you loop back to where you started, that's one
    # component of the unlink. Repeat until every strand in the unlink has been fully traversed.
    crossings = []

    # Keeps track of crossings encountered in the current component of the unlink.
    curr_component = []
    
    # Each entry in this list represents the orientation of the corresponding entry in the
    # crossing_number list. The first component is true if the crossing was found while
    # travelling up the strand; false if down. The second component is -1 if the understrand
    # goes to the left, 1 if the understrand goes to the right. See parse_braid for details.
    orientation = [[False, 0] for i in range(len(braid1) + len(braid2))]

    # Start crossing number at 1.
    crossing_num = 1
    
    # Continue iterating over strands as long as not all strands have been visited.
    while not all(used_strands):
        curr_strand = -1

        # Find a strand that has not been visited.
        for index, i in enumerate(used_strands):
            if not i:
                curr_strand = index
                break

        # Sanity check that an unused strand has been found.
        assert(curr_strand != -1)
        
        cn = crossing_num
        while True:
            # Mark the found strand as being visited.
            used_strands[curr_strand] = True
            
            # Go 'down' through the first braid word.
            curr_strand, cn = parse_braid(braid1, curr_strand, cn, crossing_number[0], orientation, curr_component, False, False)
            
            # Go 'down' through the mirror braid word in reverse.
            curr_strand, cn = parse_braid(braid2, curr_strand, cn, crossing_number[1], orientation, curr_component, True, False)
            
            # Now use the braid relation to 'cap off' in the mirror tangle to an adjacent strand.
            if curr_strand % 2 == 0:
                curr_strand += 1
            else:
                curr_strand -= 1
                
            # Now go 'up' through the mirror braid word, but not in reverse.
            curr_strand, cn = parse_braid(braid2, curr_strand, cn, crossing_number[1], orientation, curr_component, False, True)

            # Go 'up' through the first braid word.
            curr_strand, cn = parse_braid(braid1, curr_strand, cn, crossing_number[0], orientation, curr_component, True, True)

            # Mark the strand that we reached the top on as being visited.
            used_strands[curr_strand] = True
            
            # Use braid relation to cap off strand.
            if curr_strand % 2 == 0:
                curr_strand += 1
            else:
                curr_strand -= 1

            # If we cap onto a strand that has been visited, then we have completed one component of the unlink.
            if used_strands[curr_strand]: 
                break
                
            # Otherwise loop to make another pass along the strand.

        # Update what crossing number we are at.
        crossing_num = cn

        # If the component has some crossings, append to the the list that will become the Link.
        if len(curr_component) > 0:
            crossings.append(curr_component[::]) # Make a copy of curr_component (technical detail).

        # Reset the list of crossings for the next component of the unlink to traverse.
        curr_component = []

    # At this point, we have gone through, visited all the crossings, and marked the orientation of them.
    # Now all that's left is to calculate the extended part of the gauss code, as described in get_orientation.
    gauss_orientation = []
    for o in orientation:
        gauss_orientation.append(get_orientation(o)) # Get the proper orientation from crossing data
    return Link([crossings, gauss_orientation])

In [23]:
# Given the three braids in a tri-plane diagram and the bridge number, generate the three
# unlinks generated by pairwise gluing them together.
def make_links(braids, b):
    L1 = make_link(braids[0], braids[1], b)
    L2 = make_link(braids[0], braids[2], b)
    L3 = make_link(braids[1], braids[2], b)
    return L1, L2, L3

In [24]:
# This cell generates a graph out of the pairwise gluing of braids and computes the Euler Characteristic
# of said graphs. This is solely a heuristic to verify that that the braids indeed glue to unlinks.

def make_graph(braids, b):
    permutations = []
    groups = []
    B = BraidGroup(2*b) # Start with the Braid Group with 2b strands
    for i in range(3):
        braid_group = B(braids[i]) # Calculate the specific braid group with each word
        groups.append(braid_group)
        permutations.append(braid_group.permutation()) # And then generate the permutation
        # The permutation shows where the top of each strand ends up. Therefore,
        # the vertices permutation[0] and permutation[1] are the endpoints of one strand
        # in the tangle, as are permutation[2], permutation[3], etc.
        # It calculates the permutation in top-to-bottom order, so that
        # the permutation of [1,2] on 3 strands is [3,1,2]

    graph = Graph()
    graph.allow_multiple_edges(True)
    graph.add_vertices(range(2*b)) # create 2b vertices in the graph

    for i in range(0, 2*b, 2): # Go from 0 to 2b stepping by 2
        first_strand = i
        second_strand = i + 1
        for j in range(3):
            vertex1 = permutations[j][first_strand] - 1 # permutation is 1, ..., 2b, we want 0, ..., 2b-1
            vertex2 = permutations[j][second_strand] - 1
            graph.add_edge(vertex1, vertex2, j) # Add an edge connecting the two vertices; j is label, used for coloring

    plot = graph.graphplot(color_by_label=edge_colors) # Color the edges by the label (which is the tangle the edge came from)
    return graph, plot

edge_colors = {0: "red", 1: "blue", 2: "green"}
def euler_characteristic(graph):
    # Generate each of the three graphs with one of the tangles missing
    characteristic = len(graph.vertices()) # Number of vertices
    for i in range(3):
        new_graph = Graph()
        new_graph.allow_multiple_edges(True)
        new_graph.add_vertices(range(len(graph.vertices())))
        for edge in graph.edges():
            v1, v2, label = edge
            if label == i:
                continue
            else:
                new_graph.add_edge(v1, v2, label)
        characteristic += new_graph.connected_components_number() # Increment by the number of 2-cells
    return characteristic - len(graph.edges()) # Subtract number of graphs

# That the graph is orientable is yet another heuristic that the braids
# glue to become unlinks.
def is_orientable(graph):
    if graph.is_bipartite():
        return BipartiteGraph(graph)
    else:
        return None

In [37]:
# NOTE: Read the documentation for SVK_cube first. The remaining methods are helpers
# for that computation.


'''
Messy code to transform the Tietze representation of a word in the quotient group
ab/bg/ag into the corresponding word in the free group on 6b generators:
(x1, ...., xn, y1, ..., yn, z1, ..., zn)
'''
def get_tietze(word, generators):
    tietze = list(word.Tietze())
    new_tietze = []
    for num in tietze:
        g = str(word.parent().gen(abs(num)-1))
        for index, gen in enumerate(generators):
            if g == str(gen):
                new_tietze.append(-index - 1 if num < 0 else index + 1)
                break
    return new_tietze

def orientation_signs(graph, b):
    if graph is None:
        return [1 for i in range(2*b)]
    signs = []
    left = graph.left
    right = graph.right

    for vertex in graph.vertices():
        if vertex in left:
            signs.append(1)
        else:
            signs.append(-1)

    return signs

def wirtinger_relations(braid, signs, b, prefix):
    # Turn the braid into an element of the braid group, then compute
    # the permutation of the strands by its application.
    B = BraidGroup(2*b)
    braid_elem = B(braid)
    permutation = braid_elem.permutation()

    S = []
    F = FreeGroup(2*b, prefix)

    # First, each strand is mapped to a generator of the free group
    # or its inverse depending on orientation.
    for i in range(2*b):
        sign = signs[permutation[i]-1]
        S.append(F.gen(i)^sign)

    # Then according to each symbol in the braid word, the generators
    # are multiplied together according to the Wirtinger relation.
    for i in braid:
        j = abs(i)-1
        if i>0:
            S_jp1 = S[j]
            S_j = S[j]*S[j+1]*S[j]^-1

        if i<0:
            S_j = S[j+1]
            S_jp1 = S[j+1]^-1*S[j]*S[j+1]

        S[j] = S_j
        S[j+1] = S_jp1

    return S, F

# This computes the coproduct from the spine of the tri-plane diagram to the
# complement of the unlink found by gluing tangles together.
def coproduct(group1, relations1, group2, relations2, b):
    # The coproduct starts out as the free group generated by all generators.
    fg = FreeGroup(group1.gens() + group2.gens())

    # Add new relations identifying corresponding relations together.
    quotient = [fg(relations1[i])*fg(relations2[i])^-1 for i in range(len(relations1))]
    
    # Since this is the coproduct for the unlink complement, we must also add the relation
    # that adjacent generators (from the spine of the tri-plane diagram) are teh same.
    quotient = quotient + [fg.gen(2*i)*fg.gen(2*i+1)^-1 for i in range(2*b)]
    coproduct = fg/quotient
    return coproduct

# This computes the coproduct from a pair of two unlinks (which intersect in a
# single tangle of the tri-plane diagram) to the overall 2-knot complement.
#
# INPUT:
#
#   - intersection -- The fundamental group of the intersection for SVK Theorem.
#   - X1 -- The fundamental group of the first space for SVK Theorem.
#   - X1_map -- An isomorphism from X1 to a simplified representation.
#   - X2 -- The fundamental group of the second space for SVK Theorem.
#   - X2_map -- An isomorphism from X2 to a simplified representation.
#   - fg -- A free group on 8b generators, which will be quotiented by relations to yield final answer.
#   - offset -- Generators in X1 must map to the first 4b generators of fg, while
#               generators in X2 must map to the last 4b generators of fg. This offset,
#               which is always 4b, allows the mapping to happen.
def second_coproduct(intersection, X1, X1_map, X2, X2_map, fg, offset):
    # fg is freegroup on 8*b generators
    quotient = []
    
    for gen in intersection.gens():
        # For each generator in the intersection, compute what that generator
        # gets mapped to within X1 and X2. 
        # we want w1 to map to the first half of the generators and
        # w2 to shift to the second half of the generators.
        w1 = get_tietze(X1_map(gen), list(X1.gens()))
        w2 = get_tietze(X2_map(gen), list(X2.gens()))
        # Shifts generators by positive or negative amount depending on if generator is inverse
        w2 = [i + offset if i > 0 else i - offset for i in w2] # Shift to correct generators
        quotient.append(fg(w1)*fg(w2)^-1)
        
    for rel in X1.relations():
        # Add relationships from the first space
        quotient.append(fg(rel.Tietze()))
        
    for rel in X2.relations():
        fg_rel = rel.Tietze()
        # Add relationships for the second space, but once again shifted to the
        # second half of generators
        fg_rel = [i + offset if i > 0 else i - offset for i in fg_rel]
        quotient.append(fg(fg_rel))
        
    return fg/quotient

# Starting with the three tangles represented as braids words, this method uses the
# Seifert-Van Kampen Theorem to compute the fundamental group of the knotted 2-sphere.
# This is done using an iterative approach, with the name coming from how the resulting diagram
# looks like a cube:
#
# 
#           \pi(H_\beta) ------->\pi(X_1)
#           /  |                 /
#          /   |                / |
#         /    |               /  |
#   \pi(D) ------------> \pi(H_\alpha)
#        |     |           |      |
#        |   \pi(X_2) ------> \pi(X)
#        |   /             |   /
#        |  /              |  /
#        | /               | /
#   \pi(H_\gamma) ------> \pi(X_3)
#
# It starts with D, which is the S^2 "spine" of the tri-plane diagram without the 2b points where
# the tangles intersect it. It builds from there to H_i, which is the ball B^3 that is the complement
# of the unlinks found by gluing together a tangle with the mirror image of another. Finally,
# those balls are used to compute the fundamental group for the knot complement.
def SVK_cube(braids, num_b):
    # The prefixes for the generators of each fundamental group.
    prefixes = ('x','y','z')

    # The relations defining each group.
    relations = []

    # The Sage Group objects.
    groups = []

    graph, _ = make_graph(braids, num_b)
    graph = is_orientable(graph)
    signs = orientation_signs(graph, num_b)

    for i in range(3):
        # For each tangle, compute the fundamental group using Wirtinger relations.
        relation, group = wirtinger_relations(braids[i], signs, num_b, prefixes[i])
        relations.append(relation)
        groups.append(group)
        
    a, b, g = groups

    # Use coproduct to compute fundamental groups using SvK Theorem.
    ab = coproduct(a, relations[0], b, relations[1], num_b)
    X_1 = ab.simplification_isomorphism() # \pi_1(X_1)
    ag = coproduct(a, relations[0], g, relations[2], num_b)
    X_3 = ag.simplification_isomorphism() # \pi_1(X_3)
    bg = coproduct(b, relations[1], g, relations[2], num_b)
    X_2 = bg.simplification_isomorphism() # \pi_1(X_2)

    # These groups are the generators for the final answer, e.g.
    # the final fundamental group without the relations. The 8b generators
    # come from the glued unlinks each having 4b generators (2 generators for each strand)
    # and two of these glued unlinks coming together to form the overall knot complement.    
    a_fg = FreeGroup(8*num_b, 'a')
    b_fg = FreeGroup(8*num_b, 'b')
    g_fg = FreeGroup(8*num_b, 'g')

    # Use coproducts again to compute three different answers for the same fundamental group.
    # These coproducts go from intersection (single tangle) through each glued unlink to the overall 2-knot.
    final_answer1 = second_coproduct(a, ab, X_1, ag, X_3, a_fg, 4*num_b)
    final_answer2 = second_coproduct(g, bg, X_2, ag, X_3, g_fg, 4*num_b)
    final_answer3 = second_coproduct(b, ab, X_1, bg, X_2, b_fg, 4*num_b)

    return [final_answer1, final_answer2, final_answer3]

def alexander_matrix(f_group, t):
    return f_group.alexander_matrix([t for i in f_group.gens()])

In [41]:
# The Polynomial Ring that Alexander Invariants come from.
Q.<t> = LaurentPolynomialRing(ZZ, 1)

def alexander_invariant(braid, n):
    # Compute the three different fundamental groups (which should all be the same).
    g1, g2, g3 = SVK_cube(braid, n)
    invariants = []
    for g in [g1, g2, g3]:
        # For each representation of the fundamental group, compute its Alexander Matrix.
        a = alexander_matrix(g.simplified(), t)

        # Compute the minors of the matrix.
        s = min(a.nrows(), a.ncols()) - 1 if a.nrows() == a.ncols() else min(a.nrows(), a.ncols())
        minors = a.minors(s)

        # Find the ideal spanned by those minors, then take its Groebner basis for a minimal representation.
        I = Q.ideal(minors)
        invariants.append(I.groebner_basis())
    return invariants

In [42]:
# Spun Trefoil
spun_trefoil_braids = [[4,5,6,7,2,3,4,5,6,7],[2,3,4,5,6,7], [2, 3, 4, 5, 6, 7, -7, -7, -7, 1, 1, 1, 3, -5, -4]]
# 2-twist spun trefoil
two_twist_spun_trefoil_braids = [[4,5,6,7,5,6,-2,-2,-2,-3,-2,-3,-2,-3,-2,-3,-2,-3,-2,-3,-2], [4,5,-2,-2,-2],[-2,-2,-2,4,3,5,4,6,7]]

In [43]:
print(alexander_invariant(spun_trefoil_braids, 4))
print(alexander_invariant(two_twist_spun_trefoil_braids, 4))

[(t^2 - t + 1,), (t^2 - t + 1,), (t^2 - t + 1,)]
[(t + 1, 3), (t + 1, 3), (t + 1, 3)]
